In [1]:
import numpy as np
import argparse
import cv2 as cv
import subprocess
import time
import os
from yolo_utils import infer_image, show_image

FLAGS = []
f_num=0
frame_no =0
o_cnt=0
time=0


if __name__ == '__main__':
    parser = argparse.ArgumentParser()

    parser.add_argument('-m', '--model-path',
        type=str,
        default='./yolov3-coco/',
        help='The directory where the model weights and \
              configuration files are.')

    parser.add_argument('-w', '--weights',
        type=str,
        default='./yolov3-coco/yolov3.weights',
        help='Path to the file which contains the weights \
                for YOLOv3.')

    parser.add_argument('-cfg', '--config',
        type=str,
        default='./yolov3-coco/yolov3.cfg',
        help='Path to the configuration file for the YOLOv3 model.')

    parser.add_argument('-i', '--image-path',
        type=str,
        help='The path to the image file')

    parser.add_argument('-v', '--video-path',
        type=str,
        default="InputFiles/video1.mp4",
        help='The path to the video file')


    parser.add_argument('-vo', '--video-output-path',
        type=str,
        default='InputFiles/output.avi',
        help='The path of the output video file')

    parser.add_argument('-l', '--labels',
        type=str,
        default='./yolov3-coco/coco-labels',
        help='Path to the file having the \
                    labels in a new-line seperated way.')

    parser.add_argument('-c', '--confidence',
        type=float,
        default=0.5,
        help='The model will reject boundaries which has a \
                probabiity less than the confidence value. \
                default: 0.5')

    parser.add_argument('-th', '--threshold',
        type=float,
        default=0.3,
        help='The threshold to use when applying the \
                Non-Max Suppresion')

    parser.add_argument('--download-model',
        type=bool,
        default=False,
        help='Set to True, if the model weights and configurations \
                are not present on your local machine.')

    parser.add_argument('-t', '--show-time',
        type=bool,
        default=False,
        help='Show the time taken to infer each image.')

    FLAGS, unparsed = parser.parse_known_args()

    # Download the YOLOv3 models if needed
    if FLAGS.download_model:
        subprocess.call(['./yolov3-coco/get_model.sh'])

    # Get the labels
    labels = open(FLAGS.labels).read().strip().split('\n')

    # Intializing colors to represent each label uniquely
    colors = np.random.randint(0, 255, size=(len(labels), 3), dtype='uint8')

    # Load the weights and configutation to form the pretrained YOLOv3 model
    net = cv.dnn.readNetFromDarknet(FLAGS.config, FLAGS.weights)

    # Get the output layer names of the model
    layer_names = net.getLayerNames()
    layer_names = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
        

    if FLAGS.video_path:
        # Read the video
        try:
            vid = cv.VideoCapture(FLAGS.video_path)
            height, width = None, None
            writer = None
        except:
            raise 'Video cannot be loaded!\n\
                               Please check the path provided!'

        finally:
            while True:
                grabbed, frame = vid.read()

                # Checking if the complete video is read
                if not grabbed:
                    break

                if width is None or height is None:
                    height, width = frame.shape[:2]

                frame, box, _, _, id = infer_image(net, layer_names, height, width, frame, colors, labels, FLAGS)

                if writer is None:
                    # Initialize the video writer
                    fourcc = cv.VideoWriter_fourcc(*"MJPG")
                    writer = cv.VideoWriter(FLAGS.video_output_path, fourcc, 30, 
                                    (frame.shape[1], frame.shape[0]), True)


                writer.write(frame)
                if o_cnt<len(id):
                    f_num=frame
                    o_cnt=len(id)
                    f_cnt=frame_no 
                    time=vid.get(cv.CAP_PROP_POS_MSEC)
                frame_no += 1
            print ("Done")
            writer.release()
            vid.release()


   

Done


In [2]:
##Exporting Images with max number of counts of object in keyframe

name = './output/' + str(time/1000) + '.jpg'
print ('Creating File ' + str(time/1000) + '.jpg')

# writing the extracted images
cv.imwrite(name, f_num)

print("Object Count is "+str(o_cnt))

Creating File 8.933333333333334.jpg
Object Count is 11
